# auditd

## 설정 파일

- `/etc/audit` 디렉토리
  - `/etc/audit/auditd.conf` 설정 파일
  - `/etc/audit/audit.rules` 규칙 파일
    - `/etc/audit/rules.d` 규칙 디렉토리 아래의 규칙 파일에서 자동 생성
  - `/etc/audit/rules.d` 규칙 디렉토리

- `/etc/audit/auditd.conf` 설정 파일의 예시

    ```bash
    [root@localhost ~]# cat /etc/audit/auditd.conf 
    #
    # This file controls the configuration of the audit daemon
    #

    local_events = yes
    write_logs = yes
    log_file = /var/log/audit/audit.log
    log_group = root
    log_format = RAW
    flush = INCREMENTAL_ASYNC
    freq = 50
    max_log_file = 8
    num_logs = 5
    priority_boost = 4
    disp_qos = lossy
    dispatcher = /sbin/audispd
    name_format = NONE
    ##name = mydomain
    max_log_file_action = ROTATE
    space_left = 75
    space_left_action = SYSLOG
    verify_email = yes
    action_mail_acct = root
    admin_space_left = 50
    admin_space_left_action = SUSPEND
    disk_full_action = SUSPEND
    disk_error_action = SUSPEND
    use_libwrap = yes
    ##tcp_listen_port = 60
    tcp_listen_queue = 5
    tcp_max_per_addr = 1
    ##tcp_client_ports = 1024-65535
    tcp_client_max_idle = 0
    enable_krb5 = no
    krb5_principal = auditd
    ##krb5_key_file = /etc/audit/audit.key
    distribute_network = no
    ```

- `/etc/audit/audit.rules` 규칙 파일의 예시

    ```bash
    ## This file is automatically generated from /etc/audit/rules.d
    -D
    -b 8192
    -f 1
    ```

## 도구

| 명령 도구 | 역할 |
|-|-|
| `auditctl`   | 상태 출력, 설정 변경 및 임시 규칙 생성 및 삭제 | 
| `augenrules` | 영구 규칙 생성 |
| `ausearch`   | 로그 검색 |
| `aureport`   | 로그 포맷팅 |

### 상태 출력

- `auditctl -s` 명령으로 현재 상태 출력

```bash
$ auditctl -s
enabled 1
failure 1
pid 714
rate_limit 0
backlog_limit 8192
lost 0
backlog 0
loginuid_immutable 0 unlocked
```

- enabled 1
  - 감사(audit) 기능 활성화 여부
  - 1 → audit 활성 상태
  - 0 → audit 비활성
- failure 1
  - audit 시스템이 실패했을 때 동작 모드
  - 0 → ignore (무시)
  - 1 → printk (커널 로그에 기록만)
  - 2 → panic (시스템 중단)
  - 현재는 printk 상태 → audit 실패 시에도 시스템은 계속 동작
- pid 714
  - 현재 auditd 데몬의 프로세스 ID
- rate_limit 0
  - 초당 이벤트 로깅 제한
  - 0이면 무제한
  - 값이 설정되어 있으면 초당 로그 수 제한(부하 제어용)
- backlog_limit 8192
  - 커널에서 audit 이벤트를 유저 공간으로 보내기 전에 임시 저장하는 큐 크기
  - 이벤트가 몰릴 때 버퍼 역할 (8192면 비교적 큰 값)
- lost 0
  - 유실된 audit 이벤트 개수
  - 0이므로 현재 이벤트 유실 없음
- backlog 0
  - 현재 커널 큐에 쌓여 있는 이벤트 수
  - 0이면 처리 지연 없음
- loginuid_immutable 0 unlocked
  - loginuid(로그인 사용자 ID) 변경 가능 여부
  - 일반적으로 보안 강화를 위해 한 번 로그인하면 변경 불가능하도록 locked가 권장됨
  - 현재는 unlocked → 필요하다면 보안 정책에 맞게 echo 1 > /proc/sys/kernel/audit_loginuid_immutable 로 잠글 수 있음

### 설정 변경

```bash
    -e [0..2]           Set enabled flag
    -f [0..2]           Set failure flag
    -r <rate>           Set limit in messages/sec (0=none)
    -b <backlog>        Set max number of outstanding audit buffers
```

### 임시 규칙 생성

- `auditctl -w` 명령으로 임시 규칙 생성
  - 자동 생성되는 /etc/audit/audit.rules 파일에만 추가
  - 리부팅되면 없어짐


    ```bash
    auditctl -w <감시할 파일> -p <권한> -k <키 문자열>
    ```

### 규칙 출력

- `auditctl -l` 명령으로 규칙 출력

### 규칙 삭제

- `auditctl -D`: 모든 규칙 삭제
- `auditctl -W <과거 규칙>`: 해당하는 규칙 삭제

### 예시

```bash
[root@localhost ~]# auditctl -w /root -p rwxa -k root_home

[root@localhost ~]# auditctl -l
-w /root -p rwxa -k root_home

[root@localhost ~]# tail /var/log/audit/audit.log 
type=SERVICE_STOP msg=audit(1755491189.148:254): pid=1 uid=0 auid=4294967295 ses=4294967295 subj=system_u:system_r:init_t:s0 msg='unit=NetworkManager-dispatcher comm="systemd" exe="/usr/lib/systemd/systemd" hostname=? addr=? terminal=? res=success'
type=SERVICE_START msg=audit(1755491457.156:255): pid=1 uid=0 auid=4294967295 ses=4294967295 subj=system_u:system_r:init_t:s0 msg='unit=NetworkManager-dispatcher comm="systemd" exe="/usr/lib/systemd/systemd" hostname=? addr=? terminal=? res=success'
type=SERVICE_STOP msg=audit(1755491467.132:256): pid=1 uid=0 auid=4294967295 ses=4294967295 subj=system_u:system_r:init_t:s0 msg='unit=NetworkManager-dispatcher comm="systemd" exe="/usr/lib/systemd/systemd" hostname=? addr=? terminal=? res=success'
type=SERVICE_START msg=audit(1755491594.663:257): pid=1 uid=0 auid=4294967295 ses=4294967295 subj=system_u:system_r:init_t:s0 msg='unit=NetworkManager-dispatcher comm="systemd" exe="/usr/lib/systemd/systemd" hostname=? addr=? terminal=? res=success'
type=SERVICE_STOP msg=audit(1755491605.146:258): pid=1 uid=0 auid=4294967295 ses=4294967295 subj=system_u:system_r:init_t:s0 msg='unit=NetworkManager-dispatcher comm="systemd" exe="/usr/lib/systemd/systemd" hostname=? addr=? terminal=? res=success'
type=SERVICE_START msg=audit(1755491702.000:259): pid=1 uid=0 auid=4294967295 ses=4294967295 subj=system_u:system_r:init_t:s0 msg='unit=NetworkManager-dispatcher comm="systemd" exe="/usr/lib/systemd/systemd" hostname=? addr=? terminal=? res=success'
type=SERVICE_STOP msg=audit(1755491712.139:260): pid=1 uid=0 auid=4294967295 ses=4294967295 subj=system_u:system_r:init_t:s0 msg='unit=NetworkManager-dispatcher comm="systemd" exe="/usr/lib/systemd/systemd" hostname=? addr=? terminal=? res=success'
type=CONFIG_CHANGE msg=audit(1755491842.927:261): auid=0 ses=5 subj=unconfined_u:unconfined_r:unconfined_t:s0-s0:c0.c1023 op=add_rule key="root_home" list=4 res=1
type=SERVICE_START msg=audit(1755491966.130:262): pid=1 uid=0 auid=4294967295 ses=4294967295 subj=system_u:system_r:init_t:s0 msg='unit=NetworkManager-dispatcher comm="systemd" exe="/usr/lib/systemd/systemd" hostname=? addr=? terminal=? res=success'
type=SERVICE_STOP msg=audit(1755491976.130:263): pid=1 uid=0 auid=4294967295 ses=4294967295 subj=system_u:system_r:init_t:s0 msg='unit=NetworkManager-dispatcher comm="systemd" exe="/usr/lib/systemd/systemd" hostname=? addr=? terminal=? res=success'

[root@localhost ~]# touch test1

[root@localhost ~]# tail /var/log/audit/audit.log 
type=SERVICE_START msg=audit(1755491702.000:259): pid=1 uid=0 auid=4294967295 ses=4294967295 subj=system_u:system_r:init_t:s0 msg='unit=NetworkManager-dispatcher comm="systemd" exe="/usr/lib/systemd/systemd" hostname=? addr=? terminal=? res=success'
type=SERVICE_STOP msg=audit(1755491712.139:260): pid=1 uid=0 auid=4294967295 ses=4294967295 subj=system_u:system_r:init_t:s0 msg='unit=NetworkManager-dispatcher comm="systemd" exe="/usr/lib/systemd/systemd" hostname=? addr=? terminal=? res=success'
type=CONFIG_CHANGE msg=audit(1755491842.927:261): auid=0 ses=5 subj=unconfined_u:unconfined_r:unconfined_t:s0-s0:c0.c1023 op=add_rule key="root_home" list=4 res=1
type=SERVICE_START msg=audit(1755491966.130:262): pid=1 uid=0 auid=4294967295 ses=4294967295 subj=system_u:system_r:init_t:s0 msg='unit=NetworkManager-dispatcher comm="systemd" exe="/usr/lib/systemd/systemd" hostname=? addr=? terminal=? res=success'
type=SERVICE_STOP msg=audit(1755491976.130:263): pid=1 uid=0 auid=4294967295 ses=4294967295 subj=system_u:system_r:init_t:s0 msg='unit=NetworkManager-dispatcher comm="systemd" exe="/usr/lib/systemd/systemd" hostname=? addr=? terminal=? res=success'
type=SYSCALL msg=audit(1755491984.543:264): arch=c000003e syscall=2 success=yes exit=3 a0=7ffdd04d06f6 a1=941 a2=1b6 a3=7ffdd04ce720 items=2 ppid=2367 pid=2735 auid=0 uid=0 gid=0 euid=0 suid=0 fsuid=0 egid=0 sgid=0 fsgid=0 tty=pts0 ses=5 comm="touch" exe="/usr/bin/touch" subj=unconfined_u:unconfined_r:unconfined_t:s0-s0:c0.c1023 key="root_home"
type=CWD msg=audit(1755491984.543:264):  cwd="/root"
type=PATH msg=audit(1755491984.543:264): item=0 name="/root" inode=100663361 dev=fd:00 mode=040550 ouid=0 ogid=0 rdev=00:00 obj=system_u:object_r:admin_home_t:s0 objtype=PARENT cap_fp=0000000000000000 cap_fi=0000000000000000 cap_fe=0 cap_fver=0
type=PATH msg=audit(1755491984.543:264): item=1 name="test1" inode=100806597 dev=fd:00 mode=0100644 ouid=0 ogid=0 rdev=00:00 obj=unconfined_u:object_r:admin_home_t:s0 objtype=CREATE cap_fp=0000000000000000 cap_fi=0000000000000000 cap_fe=0 cap_fver=0
type=PROCTITLE msg=audit(1755491984.543:264): proctitle=746F756368007465737431

[root@localhost ~]# ausearch -k root_home
----
time->Mon Aug 18 00:37:22 2025
type=CONFIG_CHANGE msg=audit(1755491842.927:261): auid=0 ses=5 subj=unconfined_u:unconfined_r:unconfined_t:s0-s0:c0.c1023 op=add_rule key="root_home" list=4 res=1
----
time->Mon Aug 18 00:39:44 2025
type=PROCTITLE msg=audit(1755491984.543:264): proctitle=746F756368007465737431
type=PATH msg=audit(1755491984.543:264): item=1 name="test1" inode=100806597 dev=fd:00 mode=0100644 ouid=0 ogid=0 rdev=00:00 obj=unconfined_u:object_r:admin_home_t:s0 objtype=CREATE cap_fp=0000000000000000 cap_fi=0000000000000000 cap_fe=0 cap_fver=0
type=PATH msg=audit(1755491984.543:264): item=0 name="/root" inode=100663361 dev=fd:00 mode=040550 ouid=0 ogid=0 rdev=00:00 obj=system_u:object_r:admin_home_t:s0 objtype=PARENT cap_fp=0000000000000000 cap_fi=0000000000000000 cap_fe=0 cap_fver=0
type=CWD msg=audit(1755491984.543:264):  cwd="/root"
type=SYSCALL msg=audit(1755491984.543:264): arch=c000003e syscall=2 success=yes exit=3 a0=7ffdd04d06f6 a1=941 a2=1b6 a3=7ffdd04ce720 items=2 ppid=2367 pid=2735 auid=0 uid=0 gid=0 euid=0 suid=0 fsuid=0 egid=0 sgid=0 fsgid=0 tty=pts0 ses=5 comm="touch" exe="/usr/bin/touch" subj=unconfined_u:unconfined_r:unconfined_t:s0-s0:c0.c1023 key="root_home"
----
time->Mon Aug 18 01:01:01 2025
type=PROCTITLE msg=audit(1755493261.312:279): proctitle=2F7573722F7362696E2F616E6163726F6E002D73
type=PATH msg=audit(1755493261.312:279): item=0 name="./" inode=100663361 dev=fd:00 mode=040550 ouid=0 ogid=0 rdev=00:00 obj=system_u:object_r:admin_home_t:s0 objtype=NORMAL cap_fp=0000000000000000 cap_fi=0000000000000000 cap_fe=0 cap_fver=0
type=CWD msg=audit(1755493261.312:279):  cwd="/root"
type=SYSCALL msg=audit(1755493261.312:279): arch=c000003e syscall=2 success=yes exit=3 a0=55a157e9b148 a1=0 a2=55a159fa2020 a3=2 items=1 ppid=2864 pid=2868 auid=0 uid=0 gid=0 euid=0 suid=0 fsuid=0 egid=0 sgid=0 fsgid=0 tty=(none) ses=6 comm="anacron" exe="/usr/sbin/anacron" subj=system_u:system_r:system_cronjob_t:s0-s0:c0.c1023 key="root_home"

[root@localhost ~]# ausearch -k root_home | aureport -f

File Report
===============================================
# date time file syscall success exe auid event
===============================================
1. 08/18/2025 00:39:44 test1 2 yes /usr/bin/touch 0 264
2. 08/18/2025 01:01:01 ./ 2 yes /usr/sbin/anacron 0 279

[root@localhost ~]# auditctl -W /root -p rwxa -k root_home

[root@localhost ~]# auditctl -l
No rules

```

## 영구 규칙 설정 및 삭제

- 규칙 생성
  - `augenrules` 명령으로 `/etc/audit/rules.d` 규칙 디렉토리에 `.rules` 확장자를 가진 텍스트 파일 생성
- 규칙 삭제
  - `/etc/audit/rules.d` 규칙 디렉토리에서 해당 파일 삭제 후 `augenrules --load` 실행